In [44]:
import pandas as pd
import numpy as np




In [45]:
credits=pd.read_csv("tmdb_5000_credits.csv")
movies=pd.read_csv('tmdb_5000_movies.csv')
movies.head(2)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [46]:
movies.head(1)

movies=movies.merge(credits,on='title' )





In [47]:
credits.head(1)['cast'].values
movies.head(2)
movies=movies[['genres','keywords','overview','title','cast','crew','id']]



In [48]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

        


In [49]:
import ast
movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)



In [50]:
def convert(obj):
        
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
   
        
        
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            
            
            break
    return L

    


In [51]:
movies['cast']=movies['cast'].apply(convert)





In [52]:
def get_director(obj):
    L = []
    
    # If obj is already a list, use it directly
    if isinstance(obj, list):
        data = obj
    # If obj is a string, evaluate it
    elif isinstance(obj, str):
        try:
            data = ast.literal_eval(obj)
        except (ValueError, SyntaxError):
            return L
    else:
        return L
    
    for i in data:
        if i.get('job') == 'Director':
            L.append(i['name'])
            break   
    return L

    return []



In [53]:
movies['crew']=movies['crew'].apply(get_director)

movies.isnull().sum()

In [54]:

movies=movies.dropna()



In [55]:
movies.isnull().sum()


genres      0
keywords    0
overview    0
title       0
cast        0
crew        0
id          0
dtype: int64

In [56]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [57]:
movies.head(1)

,genres,keywords,overview,title,cast,crew,id
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],19995


In [58]:

movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in  x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in  x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in  x])
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in  x])


In [59]:
movies['tag']=movies['crew']+movies['keywords']+movies['cast']+movies['genres']+movies['overview']

In [60]:
movies=movies[['tag','title','id']]

In [61]:
movies['tag']=movies['tag'].apply(lambda x:" ".join(x))

In [62]:
movies['tag'].values[0]

'JamesCameron cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver Action Adventure Fantasy ScienceFiction In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [63]:
movies['tag']=movies['tag'].apply( lambda x:x.lower())

In [64]:
!pip install nl
import nltk


Defaulting to user installation because normal site-packages is not writeable


In [65]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()


In [66]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)    
        
        
    

In [67]:
movies['tag']=movies['tag'].apply(stem)
movies['tag'].values[0]

'jamescameron cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav action adventur fantasi sciencefict in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization.'

In [68]:
from sklearn.feature_extraction.text import CountVectorizer

In [69]:
cv=CountVectorizer(max_features=5000,stop_words="english")

In [70]:
vectors=cv.fit_transform(movies['tag']).toarray()
                 

In [71]:
vectors


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [72]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [73]:
movies.shape


(4806, 3)

In [74]:
#now using cosine distance rather than euclidian because it fails in higher dimension

from sklearn.metrics.pairwise import cosine_similarity






In [75]:
similarity=cosine_similarity(vectors)
similarity.shape

(4806, 4806)

In [76]:
similarity[1]
movies['title'][1]


"Pirates of the Caribbean: At World's End"

In [77]:

movies[movies['title']=="Pirates of the Caribbean: At World's End"].index[0]
sorted(similarity[1],reverse=True)[0]
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]







[(1214, 0.28676966733820225),
 (2405, 0.26901379342448517),
 (3728, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.2503866978335957)]

In [78]:
def recommend(movie):
    #index of whole data set not just so we did masking

    movie_index=movies[movies['title']==movie].index[0]
    distances=similarity[movie_index]
    #we take the firs 9 movie which are similar to the user choice and sorted on the basis of similarity score
    
    movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:10]
    for i in movie_list:
        print(movies.iloc[i[0]].title)
        

    
        
    

In [79]:
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.
Battle: Los Angeles
Predators
Small Soldiers
Jupiter Ascending


In [80]:
import pickle 
pickle.dump(similarity,open('similarity.pkl','wb'))
    


In [82]:
movies



,tag,title,id
0,jamescameron cultureclash futur spacewar space...,Avatar,19995
1,goreverbinski ocean drugabus exoticisland east...,Pirates of the Caribbean: At World's End,285
2,sammend spi basedonnovel secretag sequel mi6 b...,Spectre,206647
3,christophernolan dccomic crimefight terrorist ...,The Dark Knight Rises,49026
4,andrewstanton basedonnovel mar medallion space...,John Carter,49529
...,...,...,...
4804,robertrodriguez unitedstates–mexicobarri leg a...,El Mariachi,9367
4805,edwardburn edwardburn kerrybishé marshadietlei...,Newlyweds,72766
4806,scottsmith date loveatfirstsight narrat invest...,"Signed, Sealed, Delivered",231617
4807,danielhsia danielhenney elizacoup billpaxton w...,Shanghai Calling,126186
